In [6]:
import pandas as pd
import numpy as np

In [7]:
dy = pd.read_csv('combined_discovery_yield.csv')
dy = dy[dy.Variable!="pchembl"]
dy

,testing batch,Total Ideal Compounds,Low Error Count,Discovery Yield,Standardized Discovery Yield,Target,Variable,Method,evalt
0,1,21.0,14.0,0.666667,2.539683,target_CHEMBL240,logp,RF,sorted_fcv
1,2,37.0,23.0,0.621622,1.344047,target_CHEMBL240,logp,RF,sorted_fcv
2,3,24.0,8.0,0.333333,1.111111,target_CHEMBL240,logp,RF,sorted_fcv
3,4,43.0,18.0,0.418605,0.778799,target_CHEMBL240,logp,RF,sorted_fcv
4,5,44.0,23.0,0.522727,0.950413,target_CHEMBL240,logp,RF,sorted_fcv
...,...,...,...,...,...,...,...,...,...
337,6,80.0,35.0,0.437500,0.612500,target_CHEMBL279,NaN,MLP,scaffold_CV
338,7,94.0,44.0,0.468085,0.557718,target_CHEMBL279,NaN,MLP,scaffold_CV
339,8,80.0,40.0,0.500000,0.700000,target_CHEMBL279,NaN,MLP,scaffold_CV
340,9,79.0,40.0,0.506329,0.717834,target_CHEMBL279,NaN,MLP,scaffold_CV


In [8]:
dy.columns

Index(['testing batch', 'Total Ideal Compounds', 'Low Error Count',
       'Discovery Yield', 'Standardized Discovery Yield', 'Target', 'Variable',
       'Method', 'evalt'],
      dtype='object')

In [9]:
import pandas as pd
from scipy.stats import ttest_ind

# Group by and calculate mean and standard deviation
grouped = dy.groupby(["Target", "Method", "evalt"])["Discovery Yield"].agg(['mean', 'std']).reset_index()

# Combine mean and standard deviation into a single string
grouped["mean_std"] = grouped.apply(lambda row: f"{row['mean']:.2f} ± {row['std']:.2f}", axis=1)

# Drop the separate mean and std columns
grouped = grouped.drop(columns=["mean", "std"])

# Rename the column for clarity
dy_summary = grouped.rename(columns={"mean_std": "Discovery Yield (mean ± std)"})
display(dy_summary)

# Function to perform pairwise t-tests and find the statistically highest Discovery Yield
def find_statistically_highest(dy):
    results = []

    # Group by Target and Method
    for (target, method), group in dy.groupby(['Target', 'Method']):
        evalt_groups = group['evalt'].unique()
        max_mean = -float('inf')
        highest_evalt = None
        significant = False
        
        for evalt in evalt_groups:
            # Get Discovery Yield for the current evalt group
            dy_evalt = dy[(dy['Target'] == target) & (dy['Method'] == method) & (dy['evalt'] == evalt)]['Discovery Yield']
            mean_evalt = dy_evalt.mean()

            # Perform t-tests with all other evalt groups
            is_highest = True
            for other_evalt in evalt_groups:
                if evalt == other_evalt:
                    continue
                dy_other = dy[(dy['Target'] == target) & (dy['Method'] == method) & (dy['evalt'] == other_evalt)]['Discovery Yield']
                t_stat, p_value = ttest_ind(dy_evalt, dy_other)

                # Check if the current evalt group is statistically higher
                if p_value < 0.05 and mean_evalt <= dy_other.mean():
                    is_highest = False
                    break

            if is_highest and mean_evalt > max_mean:
                max_mean = mean_evalt
                highest_evalt = evalt
                significant = True

        results.append({
            'Target': target,
            'Method': method,
            'Highest evalt': highest_evalt,
            'Statistically Significant': significant,
            'Discovery Yield': f"{max_mean:.2f}"
        })

    return pd.DataFrame(results)

# Find the statistically highest Discovery Yield for each target and method
highest_discovery_yields = find_statistically_highest(dy)
display(highest_discovery_yields)


,Target,Method,evalt,Discovery Yield (mean ± std)
0,target_CHEMBL240,MLP,cv,0.70 ± 0.11
1,target_CHEMBL240,MLP,scaffold_CV,0.57 ± 0.07
2,target_CHEMBL240,MLP,sorted_fcv,0.62 ± 0.12
3,target_CHEMBL240,MLP,unsorted_fcv,0.63 ± 0.12
4,target_CHEMBL240,RF,cv,0.74 ± 0.06
5,target_CHEMBL240,RF,scaffold_CV,0.59 ± 0.14
6,target_CHEMBL240,RF,sorted_fcv,0.52 ± 0.14
7,target_CHEMBL240,RF,unsorted_fcv,0.65 ± 0.10
8,target_CHEMBL240,xgboost,cv,0.71 ± 0.10
9,target_CHEMBL240,xgboost,scaffold_CV,0.58 ± 0.10


,Target,Method,Highest evalt,Statistically Significant,Discovery Yield
0,target_CHEMBL240,MLP,cv,True,0.70
1,target_CHEMBL240,RF,cv,True,0.74
2,target_CHEMBL240,xgboost,cv,True,0.71
3,target_CHEMBL260,MLP,cv,True,0.65
4,target_CHEMBL260,RF,cv,True,0.74
5,target_CHEMBL260,xgboost,cv,True,0.73
6,target_CHEMBL279,MLP,cv,True,0.59
7,target_CHEMBL279,RF,cv,True,0.67
8,target_CHEMBL279,xgboost,cv,True,0.59


In [11]:
import pandas as pd
from scipy.stats import ttest_ind

# Load the data
ne = pd.read_csv('combined_novelty_error.csv')
ne = ne.replace({"CV": "cv"}) 

# Group by and calculate mean and standard deviation
grouped = ne.groupby(["Target", "Method", "evalt"])["Novelty Error"].agg(['mean', 'std']).reset_index()

# Combine mean and standard deviation into a single string
grouped["mean_std"] = grouped.apply(lambda row: f"{row['mean']:.2f} ± {row['std']:.2f}", axis=1)

# Drop the separate mean and std columns
grouped = grouped.drop(columns=["mean", "std"])

# Rename the column for clarity
ne_summary = grouped.rename(columns={"mean_std": "Novelty Error (mean ± std)"})
display(ne_summary)

# Function to perform pairwise t-tests and find the statistically lowest Novelty Error
def find_statistically_lowest(ne):
    results = []

    # Group by Target and Method
    for (target, method), group in ne.groupby(['Target', 'Method']):
        evalt_groups = group['evalt'].unique()
        min_mean = float('inf')
        lowest_evalt = None
        significant = False
        
        for evalt in evalt_groups:
            # Get Novelty Error for the current evalt group
            ne_evalt = ne[(ne['Target'] == target) & (ne['Method'] == method) & (ne['evalt'] == evalt)]['Novelty Error']
            mean_evalt = ne_evalt.mean()

            # Perform t-tests with all other evalt groups
            is_lowest = True
            for other_evalt in evalt_groups:
                if evalt == other_evalt:
                    continue
                ne_other = ne[(ne['Target'] == target) & (ne['Method'] == method) & (ne['evalt'] == other_evalt)]['Novelty Error']
                t_stat, p_value = ttest_ind(ne_evalt, ne_other)

                # Check if the current evalt group is statistically lower
                if p_value < 0.05 and mean_evalt >= ne_other.mean():
                    is_lowest = False
                    break

            if is_lowest and mean_evalt < min_mean:
                min_mean = mean_evalt
                lowest_evalt = evalt
                significant = True

        results.append({
            'Target': target,
            'Method': method,
            'Lowest evalt': lowest_evalt,
            'Statistically Significant': significant,
            'Novelty Error': f"{min_mean:.2f}"
        })

    return pd.DataFrame(results)

# Find the statistically lowest Novelty Error for each target and method
lowest_novelty_errors = find_statistically_lowest(ne)
display(lowest_novelty_errors)


,Target,Method,evalt,Novelty Error (mean ± std)
0,target_CHEMBL240,MLP,cv,1.28 ± 0.34
1,target_CHEMBL240,MLP,scaffold_CV,1.07 ± 0.59
2,target_CHEMBL240,MLP,sorted_fcv,1.20 ± 0.45
3,target_CHEMBL240,MLP,unsorted_fcv,1.03 ± 0.39
4,target_CHEMBL240,RF,cv,0.74 ± 0.51
5,target_CHEMBL240,RF,scaffold_CV,0.82 ± 0.80
6,target_CHEMBL240,RF,sorted_fcv,0.77 ± 0.32
7,target_CHEMBL240,RF,unsorted_fcv,0.63 ± 0.20
8,target_CHEMBL240,xgboost,cv,0.72 ± 0.49
9,target_CHEMBL240,xgboost,scaffold_CV,0.80 ± 0.82


,Target,Method,Lowest evalt,Statistically Significant,Novelty Error
0,target_CHEMBL240,MLP,unsorted_fcv,True,1.03
1,target_CHEMBL240,RF,unsorted_fcv,True,0.63
2,target_CHEMBL240,xgboost,unsorted_fcv,True,0.59
3,target_CHEMBL260,MLP,cv,True,0.67
4,target_CHEMBL260,RF,cv,True,0.76
5,target_CHEMBL260,xgboost,cv,True,0.79
6,target_CHEMBL279,MLP,scaffold_CV,True,0.87
7,target_CHEMBL279,RF,scaffold_CV,True,0.71
8,target_CHEMBL279,xgboost,scaffold_CV,True,0.71


In [13]:
import pandas as pd
from scipy.stats import ttest_ind

# Load the data
cm = pd.read_csv('combined_metrics.csv')
cm = cm[cm.Sorting_Var != "pchembl"]

# Group by and calculate mean and standard deviation
grouped = cm.groupby(["Target", "Method", "evalt"])["R2"].agg(['mean', 'std']).reset_index()

# Combine mean and standard deviation into a single string
grouped["mean_std"] = grouped.apply(lambda row: f"{row['mean']:.2f} ± {row['std']:.2f}", axis=1)

# Drop the separate mean and std columns
grouped = grouped.drop(columns=["mean", "std"])

# Rename the column for clarity
r2_summary = grouped.rename(columns={"mean_std": "R2 (mean ± std)"})
display(r2_summary)

# Function to perform pairwise t-tests and find the statistically highest R2
def find_statistically_highest(cm):
    results = []

    # Group by Target and Method
    for (target, method), group in cm.groupby(['Target', 'Method']):
        evalt_groups = group['evalt'].unique()
        max_mean = -float('inf')
        highest_evalt = None
        significant = False
        
        for evalt in evalt_groups:
            # Get R2 for the current evalt group
            r2_evalt = cm[(cm['Target'] == target) & (cm['Method'] == method) & (cm['evalt'] == evalt)]['R2']
            mean_evalt = r2_evalt.mean()

            # Perform t-tests with all other evalt groups
            is_highest = True
            for other_evalt in evalt_groups:
                if evalt == other_evalt:
                    continue
                r2_other = cm[(cm['Target'] == target) & (cm['Method'] == method) & (cm['evalt'] == other_evalt)]['R2']
                t_stat, p_value = ttest_ind(r2_evalt, r2_other)

                # Check if the current evalt group is statistically higher
                if p_value < 0.05 and mean_evalt <= r2_other.mean():
                    is_highest = False
                    break

            if is_highest and mean_evalt > max_mean:
                max_mean = mean_evalt
                highest_evalt = evalt
                significant = True

        results.append({
            'Target': target,
            'Method': method,
            'Highest evalt': highest_evalt,
            'Statistically Significant': significant,
            'R2': f"{max_mean:.2f}"
        })

    return pd.DataFrame(results)

# Find the statistically highest R2 for each target and method
highest_r2 = find_statistically_highest(cm)
display(highest_r2)


,Target,Method,evalt,R2 (mean ± std)
0,target_CHEMBL240,MLP,cv,0.58 ± 0.10
1,target_CHEMBL240,MLP,scaffold_CV,0.08 ± 0.58
2,target_CHEMBL240,MLP,sorted_fcv,-0.11 ± 0.95
3,target_CHEMBL240,MLP,unsorted_fcv,-0.09 ± 0.95
4,target_CHEMBL240,RF,cv,0.70 ± 0.07
5,target_CHEMBL240,RF,scaffold_CV,0.41 ± 0.31
6,target_CHEMBL240,RF,sorted_fcv,0.35 ± 0.46
7,target_CHEMBL240,RF,unsorted_fcv,0.48 ± 0.24
8,target_CHEMBL240,xgboost,cv,0.70 ± 0.09
9,target_CHEMBL240,xgboost,scaffold_CV,0.38 ± 0.27


,Target,Method,Highest evalt,Statistically Significant,R2
0,target_CHEMBL240,MLP,cv,True,0.58
1,target_CHEMBL240,RF,cv,True,0.70
2,target_CHEMBL240,xgboost,cv,True,0.70
3,target_CHEMBL260,MLP,cv,True,0.66
4,target_CHEMBL260,RF,cv,True,0.76
5,target_CHEMBL260,xgboost,cv,True,0.74
6,target_CHEMBL279,MLP,cv,True,0.53
7,target_CHEMBL279,RF,cv,True,0.66
8,target_CHEMBL279,xgboost,cv,True,0.62


In [15]:
import pandas as pd
from scipy.stats import ttest_ind

# Load the data
cm = pd.read_csv('combined_metrics.csv')
cm = cm[cm.Sorting_Var != "pchembl"]

# Group by and calculate mean and standard deviation
grouped = cm.groupby(["Target", "Method", "evalt"])["RMSE"].agg(['mean', 'std']).reset_index()

# Combine mean and standard deviation into a single string
grouped["mean_std"] = grouped.apply(lambda row: f"{row['mean']:.2f} ± {row['std']:.2f}", axis=1)

# Drop the separate mean and std columns
grouped = grouped.drop(columns=["mean", "std"])

# Rename the column for clarity
rmse_summary = grouped.rename(columns={"mean_std": "RMSE (mean ± std)"})
display(rmse_summary)

# Function to perform pairwise t-tests and find the statistically lowest RMSE
def find_statistically_lowest(cm):
    results = []

    # Group by Target and Method
    for (target, method), group in cm.groupby(['Target', 'Method']):
        evalt_groups = group['evalt'].unique()
        min_mean = float('inf')
        lowest_evalt = None
        significant = False
        
        for evalt in evalt_groups:
            # Get RMSE for the current evalt group
            rmse_evalt = cm[(cm['Target'] == target) & (cm['Method'] == method) & (cm['evalt'] == evalt)]['RMSE']
            mean_evalt = rmse_evalt.mean()

            # Perform t-tests with all other evalt groups
            is_lowest = True
            for other_evalt in evalt_groups:
                if evalt == other_evalt:
                    continue
                rmse_other = cm[(cm['Target'] == target) & (cm['Method'] == method) & (cm['evalt'] == other_evalt)]['RMSE']
                t_stat, p_value = ttest_ind(rmse_evalt, rmse_other)

                # Check if the current evalt group is statistically lower
                if p_value < 0.05 and mean_evalt >= rmse_other.mean():
                    is_lowest = False
                    break

            if is_lowest and mean_evalt < min_mean:
                min_mean = mean_evalt
                lowest_evalt = evalt
                significant = True

        results.append({
            'Target': target,
            'Method': method,
            'Lowest evalt': lowest_evalt,
            'Statistically Significant': significant,
            'RMSE': f"{min_mean:.2f}"
        })

    return pd.DataFrame(results)

# Find the statistically lowest RMSE for each target and method
lowest_rmse = find_statistically_lowest(cm)
display(lowest_rmse)



,Target,Method,evalt,RMSE (mean ± std)
0,target_CHEMBL240,MLP,cv,0.66 ± 0.06
1,target_CHEMBL240,MLP,scaffold_CV,0.81 ± 0.21
2,target_CHEMBL240,MLP,sorted_fcv,0.91 ± 0.30
3,target_CHEMBL240,MLP,unsorted_fcv,0.92 ± 0.25
4,target_CHEMBL240,RF,cv,0.55 ± 0.07
5,target_CHEMBL240,RF,scaffold_CV,0.69 ± 0.35
6,target_CHEMBL240,RF,sorted_fcv,0.72 ± 0.20
7,target_CHEMBL240,RF,unsorted_fcv,0.68 ± 0.07
8,target_CHEMBL240,xgboost,cv,0.55 ± 0.07
9,target_CHEMBL240,xgboost,scaffold_CV,0.71 ± 0.33


,Target,Method,Lowest evalt,Statistically Significant,RMSE
0,target_CHEMBL240,MLP,cv,True,0.66
1,target_CHEMBL240,RF,cv,True,0.55
2,target_CHEMBL240,xgboost,cv,True,0.55
3,target_CHEMBL260,MLP,cv,True,0.67
4,target_CHEMBL260,RF,cv,True,0.57
5,target_CHEMBL260,xgboost,cv,True,0.59
6,target_CHEMBL279,MLP,cv,True,0.74
7,target_CHEMBL279,RF,cv,True,0.63
8,target_CHEMBL279,xgboost,cv,True,0.67
